In [8]:
###### general
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import time

#data scrapping
import pysradb as sra #pip install pysradb
from Bio import Entrez

# data visualization

from  matplotlib import pyplot as plt
from matplotlib import cm, colors


# CCLE Project Sample DataFrame - Experiment Info and Accession IDs

In [9]:
sra_db=sra.SRAweb()
project_items= sra_db.search_sra("PRJNA523380")

In [10]:
db= pd.DataFrame(project_items)

In [11]:
db.bioproject.unique()

array(['PRJNA523380'], dtype=object)

## Cleaning 001

In [12]:
db_01= db.drop(['library_selection','library_layout','bioproject','sample_title','organism_taxid','organism_name','study_title','total_spots','total_size','run_total_spots','run_total_bases'],axis=1)

## Discovery 001

In [8]:
pd.unique( db_01.instrument)

array(['Illumina MiSeq', 'Illumina HiSeq 2500', 'Illumina HiSeq 2000'],
      dtype=object)

In [9]:
db_01.shape

(4550, 13)

In [12]:
db_01['library_strategy'].value_counts()

library_strategy
RNA-Seq             1019
Targeted-Capture     976
AMPLICON             972
Bisulfite-Seq        928
WGS                  329
WXS                  326
Name: count, dtype: int64

In [ ]:
db_01.library_strategy.unique()

array(['AMPLICON', 'RNA-Seq', 'WXS', 'Bisulfite-Seq', 'WGS',
       'Targeted-Capture'], dtype=object)

In [ ]:
db_01[db_01['library_strategy']=="AMPLICON"].shape

(972, 13)

In [ ]:
db_01[db_01['library_strategy']=="RNA-Seq"].shape

(1019, 13)

In [ ]:
db_01[db_01['library_strategy']=="WXS"].shape

(326, 13)

In [ ]:
db_01[db_01['library_strategy']=="Bisulfite-Seq"].shape


(928, 13)

In [ ]:
db_01[db_01['library_strategy']=="WGS"].shape

(329, 13)

In [ ]:
db_01[db_01['library_strategy']=="Targeted-Capture"].shape

(976, 13)

In [ ]:
db_01.biosample.unique().shape

(1065,)

In [18]:
db_01['study_accession'].value_counts()

study_accession
SRP186687    4550
Name: count, dtype: int64

In [17]:
len(pd.unique(db_01['experiment_accession']))

4550

In [15]:
db_01[db_01['biosample']=="SAMN10988035"]

,study_accession,experiment_accession,experiment_title,experiment_desc,library_name,library_strategy,library_source,sample_accession,biosample,instrument,instrument_model,instrument_model_desc,run_accession
4360,SRP186687,SRX5414209,TERTp of HCC366_LUNG,TERTp of HCC366_LUNG,TERTp-HCC366_LUNG,AMPLICON,GENOMIC,SRS4395289,SAMN10988035,Illumina MiSeq,Illumina MiSeq,ILLUMINA,SRR8615031
4274,SRP186687,SRX5414296,RNAseq of HCC366_LUNG,RNAseq of HCC366_LUNG,RNASeq-HCC366_LUNG,RNA-Seq,TRANSCRIPTOMIC,SRS4395289,SAMN10988035,Illumina HiSeq 2500,Illumina HiSeq 2500,ILLUMINA,SRR8615457
3169,SRP186687,SRX5417973,WXS of HCC366_LUNG,WXS of HCC366_LUNG,WXS-HCC366_LUNG,WXS,GENOMIC,SRS4395289,SAMN10988035,Illumina HiSeq 2000,Illumina HiSeq 2000,ILLUMINA,SRR8619114
2238,SRP186687,SRX5432664,RRBS of HCC366_LUNG,RRBS of HCC366_LUNG,RRBS-HCC366_LUNG,Bisulfite-Seq,GENOMIC,SRS4395289,SAMN10988035,Illumina HiSeq 2000,Illumina HiSeq 2000,ILLUMINA,SRR8634007
1846,SRP186687,SRX5434272,RainDance of HCC366_LUNG,RainDance of HCC366_LUNG,RainDance-HCC366_LUNG,AMPLICON,GENOMIC,SRS4395289,SAMN10988035,Illumina HiSeq 2000,Illumina HiSeq 2000,ILLUMINA,SRR8635450
301,SRP186687,SRX5455557,HC of HCC366_LUNG,HC of HCC366_LUNG,HC-HCC366_LUNG,Targeted-Capture,GENOMIC,SRS4395289,SAMN10988035,Illumina HiSeq 2000,Illumina HiSeq 2000,ILLUMINA,SRR8657908


# Sample DataFrame with Entrez info- info about disease

## Once and for all, SKIP ME and  just load the csv file on the next step  

In [ ]:
db_02=db_01.copy()

## Function Definition

In [61]:
#dict_sampleName= {}
dict_sampleName2= {}
Entrez.email='@gmail.com'#'insert your email adressses
# I used multiple email accounts to trick the requests limit
def get_sampleName(str_id):
    str_out=""
    try:
        search_result= Entrez.efetch(db= "biosample", id=str_id)
       # print(id)
        tree=ET.parse(search_result)
        root=tree.getroot()
        search_result.close()
    except:
        return ""

    str_sampleName=""
    str_tissue=""
    str_cell_line=""
    str_disease=""
    str_ethnicity=""

    for item in root.iter():
       # print(item.tag)
        if(item.tag=="Id"):
           # print(item.text)
           # print(len(item.attrib))
            if ("db_label" in item.attrib):
                str_sampleName=item.text
                #break
        if (item.tag=="Attribute"):
            #print()
            if ("tissue" in item.attrib.values()):
                str_tissue=item.text
            elif ("cell_line" in item.attrib.values()):
                str_cell_line=item.text
            elif ("disease" in item.attrib.values()):
                str_disease=item.text
            elif ("ethnicity"in item.attrib.values()):
                str_ethnicity=item.text
            


    str_out= str_sampleName+","+str_tissue+","+str_cell_line+","+str_disease+","+str_ethnicity

    return str_out #root[0][1][0].text


i=1
str_tmp=""

for item in db_01.biosample.unique():
    str_tmp=""
    if i%20==0 :
        current_time = time.strftime("%H:%M:%S", time.localtime())
        print("Elementi Dict: "+str(len(dict_sampleName2))+" ,"+current_time)
        i=1
        Entrez.email='@gmail.com'
        time.sleep(10)

    elif i%15==0:
        Entrez.email='@gmail.com'
    elif i%10==0:
        Entrez.email='@gmail.com'
    elif i%5==0:
        Entrez.email='@gmail.com'

    if not(item in dict_sampleName2.keys()):
        str_tmp=get_sampleName(item)
        if str_tmp!="":
            dict_sampleName2[item]=str_tmp
        i+=1
        time.sleep(1)

    #print(get_sampleName(item))

np.save("dict_biosample2.npy",dict_sampleName)

Elementi Dict: 19 ,22:37:24
Elementi Dict: 38 ,22:38:05
Elementi Dict: 57 ,22:38:47
Elementi Dict: 76 ,22:39:29
Elementi Dict: 95 ,22:40:15
Elementi Dict: 114 ,22:40:57
Elementi Dict: 133 ,22:41:39
Elementi Dict: 151 ,22:44:30
Elementi Dict: 170 ,22:45:19
Elementi Dict: 189 ,22:46:03
Elementi Dict: 208 ,22:46:45
Elementi Dict: 227 ,22:47:26
Elementi Dict: 246 ,22:48:09
Elementi Dict: 264 ,22:48:51
Elementi Dict: 283 ,22:49:34
Elementi Dict: 302 ,22:50:18
Elementi Dict: 321 ,22:51:01
Elementi Dict: 340 ,22:51:42
Elementi Dict: 359 ,22:53:10
Elementi Dict: 378 ,22:53:54
Elementi Dict: 397 ,22:54:37
Elementi Dict: 415 ,22:55:23
Elementi Dict: 434 ,22:56:06
Elementi Dict: 453 ,22:56:54
Elementi Dict: 472 ,22:57:38
Elementi Dict: 491 ,22:58:21
Elementi Dict: 510 ,22:59:02
Elementi Dict: 529 ,22:59:45
Elementi Dict: 548 ,23:00:42
Elementi Dict: 567 ,23:02:42
Elementi Dict: 586 ,23:03:24
Elementi Dict: 605 ,23:04:10
Elementi Dict: 624 ,23:04:52
Elementi Dict: 643 ,23:05:34
Elementi Dict: 662 

NameError: name 'dict_sampleName' is not defined

## Cleaning 002

In [62]:
serie_test= pd.Series(dict_sampleName2)

In [63]:
table_sampleName=serie_test.to_frame()

In [64]:
table_sampleName.index.name = "Sample_ID"
table_sampleName.reset_index(inplace=True)
table_sampleName=table_sampleName.set_axis(['Sample_ID', 'tmp'],axis='columns')

In [65]:
table_sampleName["Sample_Name"]=table_sampleName["tmp"].apply(lambda x: x.split(",")[0])

In [66]:
table_sampleName["tissue"]=table_sampleName["tmp"].apply(lambda x: x.split(",")[1])

In [67]:
table_sampleName["cell_line"]=table_sampleName["tmp"].apply(lambda x: x.split(",")[2])

In [68]:
table_sampleName["disease"]=table_sampleName["tmp"].apply(lambda x: x.split(",")[3])

In [69]:
table_sampleName["ethnicity"]=table_sampleName["tmp"].apply(lambda x: x.split(",")[4])

In [70]:
table_sampleName.drop("tmp", axis=1)

,Sample_ID,Sample_Name,tissue,cell_line,disease,ethnicity
0,SAMN10988035,HCC366_LUNG,lung,HCC-366,carcinoma (mixed_adenosquamous_carcinoma),Caucasian
1,SAMN10988526,HCC4006_LUNG,lung,HCC4006,carcinoma (adenocarcinoma),Caucasian
2,SAMN10988141,HCC56_LARGE_INTESTINE,large_intestine,HCC-56,carcinoma (adenocarcinoma),Asian
3,SAMN10988287,HCC78_LUNG,lung,HCC-78,carcinoma (adenocarcinoma),Caucasian
4,SAMN10989587,HOP62_LUNG,lung,HOP-62,carcinoma (adenocarcinoma),Caucasian
...,...,...,...,...,...,...
1057,SAMN10988010,CORL51_LUNG,lung,COR-L51,carcinoma (small_cell_carcinoma),Caucasian
1058,SAMN10988106,COLO818_SKIN,skin,COLO-818,malignant_melanoma,Caucasian
1059,SAMN10988202,MOLT4_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,haematopoietic_and_lymphoid_tissue,MOLT-4,lymphoid_neoplasm (acute_lymphoblastic_T_cell_...,Caucasian
1060,SAMN10987675,OC316_OVARY,ovary,OC 316,carcinoma (adenocarcinoma),Caucasian


## Save df on CSV

In [71]:
table_sampleName.to_csv("table_sampleName.csv",sep="\t")

In [ ]:
table_sampleName["tissue"].unique()

# Final Dataframe

## Load CSV  sample table and affymetrix info file

In [15]:
table_sampleName= pd.read_csv("table_sampleName.csv",sep="\t")
df_copyNumber= pd.read_csv("CCLE_SNP.Arrays.sif_2013-12-03.txt",sep="\t")
db_02=db_01.copy()

In [16]:
db_02=db_02.set_axis(['study_accession',
 'experiment_accession',
 'experiment_title',
 'experiment_desc',
 'library_name',
 'library_strategy',
 'library_source',
 'sample_accession',
 'Sample_ID',
 'instrument',
 'instrument_model',
 'instrument_model_desc',
 'run_accession'],axis="columns")

## Merge Dataframes

In [17]:
db_03=db_02.join(table_sampleName.set_index('Sample_ID'), on='Sample_ID', validate='m:1')

In [18]:
db_03.drop(["tmp"],axis=1)

,study_accession,experiment_accession,experiment_title,experiment_desc,library_name,library_strategy,library_source,sample_accession,Sample_ID,instrument,instrument_model,instrument_model_desc,run_accession,Unnamed: 0,Sample_Name,tissue,cell_line,disease
4360,SRP186687,SRX5414209,TERTp of HCC366_LUNG,TERTp of HCC366_LUNG,TERTp-HCC366_LUNG,AMPLICON,GENOMIC,SRS4395289,SAMN10988035,Illumina MiSeq,Illumina MiSeq,ILLUMINA,SRR8615031,0,HCC366_LUNG,lung,HCC-366,carcinoma (mixed_adenosquamous_carcinoma)
4361,SRP186687,SRX5414208,TERTp of HCC4006_LUNG,TERTp of HCC4006_LUNG,TERTp-HCC4006_LUNG,AMPLICON,GENOMIC,SRS4395288,SAMN10988526,Illumina MiSeq,Illumina MiSeq,ILLUMINA,SRR8615032,1,HCC4006_LUNG,lung,HCC4006,carcinoma (adenocarcinoma)
4362,SRP186687,SRX5414207,TERTp of HCC56_LARGE_INTESTINE,TERTp of HCC56_LARGE_INTESTINE,TERTp-HCC56_LARGE_INTESTINE,AMPLICON,GENOMIC,SRS4395287,SAMN10988141,Illumina MiSeq,Illumina MiSeq,ILLUMINA,SRR8615033,2,HCC56_LARGE_INTESTINE,large_intestine,HCC-56,carcinoma (adenocarcinoma)
4363,SRP186687,SRX5414206,TERTp of HCC78_LUNG,TERTp of HCC78_LUNG,TERTp-HCC78_LUNG,AMPLICON,GENOMIC,SRS4395286,SAMN10988287,Illumina MiSeq,Illumina MiSeq,ILLUMINA,SRR8615034,35,HCC78_LUNG,lung,HCC-78,carcinoma (adenocarcinoma)
4364,SRP186687,SRX5414205,TERTp of HOP62_LUNG,TERTp of HOP62_LUNG,TERTp-HOP62_LUNG,AMPLICON,GENOMIC,SRS4395285,SAMN10989587,Illumina MiSeq,Illumina MiSeq,ILLUMINA,SRR8615035,3,HOP62_LUNG,lung,HOP-62,carcinoma (adenocarcinoma)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,SRP186687,SRX5466605,WGS of SKUT1_SOFT_TISSUE,WGS of SKUT1_SOFT_TISSUE,WGS-SKUT1_SOFT_TISSUE,WGS,GENOMIC,SRS4395564,SAMN10987664,Illumina HiSeq 2000,Illumina HiSeq 2000,ILLUMINA,SRR8670775,691,SKUT1_SOFT_TISSUE,soft_tissue,SK-UT-1,leiomyosarcoma
0,SRP186687,SRX5578769,WGS of DANG_PANCREAS,WGS of DANG_PANCREAS,WGS-DANG_PANCREAS,WGS,GENOMIC,SRS4395998,SAMN10987807,Illumina HiSeq 2000,Illumina HiSeq 2000,ILLUMINA,SRR8788980,776,DANG_PANCREAS,pancreas,DAN-G,carcinoma
1,SRP186687,SRX5578768,WGS of DMS114_LUNG,WGS of DMS114_LUNG,WGS-DMS114_LUNG,WGS,GENOMIC,SRS4395964,SAMN10988190,Illumina HiSeq 2000,Illumina HiSeq 2000,ILLUMINA,SRR8788981,810,DMS114_LUNG,lung,DMS 114,carcinoma (small_cell_carcinoma)
2,SRP186687,SRX5578767,WGS of DMS273_LUNG,WGS of DMS273_LUNG,WGS-DMS273_LUNG,WGS,GENOMIC,SRS4395963,SAMN10987982,Illumina HiSeq 2000,Illumina HiSeq 2000,ILLUMINA,SRR8788982,812,DMS273_LUNG,lung,DMS 273,carcinoma (small_cell_carcinoma)


## Discovery 002

In [19]:
subGroups=['RNA-Seq' ,'WXS','WGS']
db_04=db_03.loc[db_03['library_strategy'].isin(subGroups)]

In [20]:
db_04['library_strategy'].value_counts()

library_strategy
RNA-Seq    1019
WGS         329
WXS         326
Name: count, dtype: int64

In [21]:
db_04['tissue'].value_counts()

tissue
lung                                  323
haematopoietic_and_lymphoid_tissue    279
breast                                104
large_intestine                        98
central_nervous_system                 93
skin                                   92
ovary                                  86
stomach                                74
pancreas                               68
upper_aerodigestive_tract              56
endometrium                            56
kidney                                 48
liver                                  45
soft_tissue                            43
oesophagus                             40
urinary_tract                          39
bone                                   38
autonomic_ganglia                      23
pleura                                 17
thyroid                                16
biliary_tract                          15
prostate                               15
cervix                                  2
small_intestine            

In [22]:
db_04['tissue'].value_counts().sum()


1674

In [23]:
db_04.loc[db_04['tissue'].isnull()]

,study_accession,experiment_accession,experiment_title,experiment_desc,library_name,library_strategy,library_source,sample_accession,Sample_ID,instrument,instrument_model,instrument_model_desc,run_accession,Unnamed: 0,tmp,Sample_Name,tissue,cell_line,disease


## Discovery 003

### Function Definitions

In [26]:
def get_lib_strategy(inpt_strategy,inpt_sampleName, inpt_db):
    if inpt_strategy in list(inpt_db[inpt_db["Sample_Name"]==inpt_sampleName]["library_strategy"]):
        return True
    else:
        return False

def get_lib_multiStrategy(inpt_strategy,inpt_notStrategy,inpt_sampleName, inpt_db):
    int_len=len(inpt_strategy)
    int_notLen =len(inpt_notStrategy)


    tmp_list=list(inpt_db[inpt_db["Sample_Name"]==inpt_sampleName]["library_strategy"])

    if int_notLen==0:

        for item in inpt_strategy:
            if not(item in tmp_list):
                return False

        return True
    else:
        for item in inpt_strategy:
            if not(item in tmp_list):
                return False

        for item in inpt_notStrategy:
            if item in tmp_list:
                return False

        return True


def count_libStrategy_byTissue(inpt_strategy,inpt_tissue, inpt_db):

    return inpt_db[inpt_db["tissue"]==inpt_tissue][inpt_strategy].sum()

def get_DNAffy(inpt_tissue,inpt_list):
    if inpt_tissue in inpt_list:
        return True
    else:
        return False



def get_SRR_name(inpt_strategy,inpt_series,inpt_db):
    tmp_name=inpt_series["Sample_ID"]
    flag_thereIs =bool(inpt_series[inpt_strategy])


    if flag_thereIs:

        return( inpt_db.loc[(inpt_db["Sample_ID"]==tmp_name) & (inpt_db["library_strategy"]==inpt_strategy)]["run_accession"].values[0])
    else:
        return "none"



## DataFrame Preprocessing

In [27]:
table_sampleName_1=table_sampleName.copy()

In [28]:
table_sampleName_1["RNA-Seq"]= table_sampleName_1['Sample_Name'].apply(lambda x: get_lib_strategy('RNA-Seq',x,db_04))
table_sampleName_1["WGS"]= table_sampleName_1['Sample_Name'].apply(lambda x: get_lib_strategy('WGS',x,db_04))
table_sampleName_1["WXS"]= table_sampleName_1['Sample_Name'].apply(lambda x: get_lib_strategy('WXS',x,db_04))


In [29]:
table_sampleName_1["RnaSeq+WGS+WXS"]= table_sampleName_1['Sample_Name'].apply(lambda x: get_lib_multiStrategy(['RNA-Seq','WGS','WXS'],[],x,db_04))
table_sampleName_1["RnaSeq_only"]= table_sampleName_1['Sample_Name'].apply(lambda x: get_lib_multiStrategy(['RNA-Seq'],['WGS','WXS'],x,db_04))


In [30]:
tmp_list=list(df_copyNumber["CleanCellLine"])
table_sampleName_1["DNA_AffyMtrx"]=table_sampleName_1['Sample_Name'].apply(lambda x: get_DNAffy(x,tmp_list))

In [31]:
table_sampleName_1["RnaSeq+W*S+DNAffy"]= table_sampleName_1.apply(lambda x: x["RnaSeq+WGS+WXS"] and x['DNA_AffyMtrx'],axis=1)


In [32]:
table_sampleName_1["SRR_RNA-Seq"]=table_sampleName_1.apply(lambda x: get_SRR_name("RNA-Seq",x,db_04),axis=1)
table_sampleName_1["SRR_WGS"]=table_sampleName_1.apply(lambda x: get_SRR_name("WGS",x,db_04),axis=1)
table_sampleName_1["SRR_WXS"]=table_sampleName_1.apply(lambda x: get_SRR_name("WXS",x,db_04),axis=1)

In [ ]:
## Filter table for sa

In [33]:
table_sampleName_1_toSave=table_sampleName_1[table_sampleName_1.apply(lambda x: (not x["RNA-Seq"]) and (not x["WGS"]) and (not x["WXS"]) and x['DNA_AffyMtrx'],axis=1)]#.to_csv("only_affy.txt")

In [34]:
table_sampleName_1_toSave.columns

Index(['Unnamed: 0', 'Sample_ID', 'tmp', 'Sample_Name', 'tissue', 'cell_line',
       'disease', 'RNA-Seq', 'WGS', 'WXS', 'RnaSeq+WGS+WXS', 'RnaSeq_only',
       'DNA_AffyMtrx', 'RnaSeq+W*S+DNAffy', 'SRR_RNA-Seq', 'SRR_WGS',
       'SRR_WXS'],
      dtype='object')

In [35]:
table_sampleName_1_toSave=table_sampleName_1_toSave.drop(['Unnamed: 0', 'RNA-Seq', 'WGS', 'WXS', 'RnaSeq+WGS+WXS', 'RnaSeq_only',
       'DNA_AffyMtrx', 'RnaSeq+W*S+DNAffy', 'SRR_RNA-Seq', 'SRR_WGS','SRR_WXS'],axis=1)

## Save Final DataFrame

In [36]:
table_sampleName_1_toSave.to_csv("table_sampleName_affy.csv",index=False,sep="\t")

##  Sample Table (grouped by Tissue) data

In [38]:
table_sampleName2Print=table_sampleName_1.sort_values(by="tissue").drop(['Unnamed: 0', 'Sample_ID', 'tmp', 'disease', 'RnaSeq+WGS+WXS', 'RnaSeq_only','RnaSeq+W*S+DNAffy', 'SRR_RNA-Seq', 'SRR_WGS','SRR_WXS'], axis=1).reset_index().drop("index",axis=1)

In [39]:
table_sampleName2Print.to_csv("lista_lineeCell_tipoDati.csv",index=False)

In [40]:
table_sampleName2Print

,Sample_Name,tissue,cell_line,RNA-Seq,WGS,WXS,DNA_AffyMtrx
0,CHP126_AUTONOMIC_GANGLIA,autonomic_ganglia,CHP-126,True,False,False,True
1,SKNFI_AUTONOMIC_GANGLIA,autonomic_ganglia,SK-N-FI,True,False,False,True
2,SKNAS_AUTONOMIC_GANGLIA,autonomic_ganglia,SK-N-AS,True,True,False,True
3,SHSY5Y_AUTONOMIC_GANGLIA,autonomic_ganglia,SH-SY5Y,False,False,False,True
4,NB1_AUTONOMIC_GANGLIA,autonomic_ganglia,NB-1,True,False,True,True
...,...,...,...,...,...,...,...
1060,ACCS_URINARY_TRACT,urinary_tract,ACCS,False,False,False,False
1061,RT112_URINARY_TRACT,urinary_tract,RT-112,True,True,False,True
1062,HT1376_URINARY_TRACT,urinary_tract,HT-1376,True,True,False,True
1063,HT1197_URINARY_TRACT,urinary_tract,HT-1197,True,False,False,True


In [41]:
df_4Waffles=table_sampleName_1.groupby("tissue").size()

In [42]:
df_4waffle_table=df_4Waffles.to_frame().reset_index()
df_4waffle_table=df_4waffle_table.set_axis(["tissue","samples_tot"], axis=1)
df_4waffle_table=df_4waffle_table.sort_values("samples_tot",ascending=False)
df_4waffle_table["RnaSeq_count"]= df_4waffle_table["tissue"].apply(lambda x: count_libStrategy_byTissue('RNA-Seq',x,table_sampleName_1) )
df_4waffle_table["WGS_count"]= df_4waffle_table["tissue"].apply(lambda x: count_libStrategy_byTissue('WGS',x,table_sampleName_1))
df_4waffle_table["WXS_count"]= df_4waffle_table["tissue"].apply(lambda x: count_libStrategy_byTissue('WXS',x,table_sampleName_1))
df_4waffle_table["RnaSeq+WGS+WXS"]= df_4waffle_table["tissue"].apply(lambda x: count_libStrategy_byTissue("RnaSeq+WGS+WXS",x,table_sampleName_1))
df_4waffle_table["RnaSeq_only"]= df_4waffle_table["tissue"].apply(lambda x: count_libStrategy_byTissue("RnaSeq_only",x,table_sampleName_1))
df_4waffle_table["DNA_AffyMtrx"]=df_4waffle_table["tissue"].apply(lambda x: count_libStrategy_byTissue("DNA_AffyMtrx",x,table_sampleName_1))
df_4waffle_table["RnaSeq+W*S+DNAffy"]=df_4waffle_table["tissue"].apply(lambda x: count_libStrategy_byTissue("RnaSeq+W*S+DNAffy",x,table_sampleName_1))


In [43]:
df_4waffle_table2print=df_4waffle_table.copy()
def absPercString(num1,num2):
    tmp = 100*num1/num2
    return "{} [{:.2f}%]".format(num1,tmp)
df_4waffle_table2print["RnaSeq_count"]=df_4waffle_table2print.apply(lambda x: absPercString(x["RnaSeq_count"],x["samples_tot"]),axis=1)
df_4waffle_table2print["WGS_count"]=df_4waffle_table2print.apply(lambda x: absPercString(x["WGS_count"],x["samples_tot"]),axis=1)
df_4waffle_table2print["WXS_count"]=df_4waffle_table2print.apply(lambda x: absPercString(x["WXS_count"],x["samples_tot"]),axis=1)
df_4waffle_table2print["DNA_AffyMtrx"]=df_4waffle_table2print.apply(lambda x: absPercString(x["DNA_AffyMtrx"],x["samples_tot"]),axis=1)

In [44]:
tmp_list=list(df_4waffle_table.agg({"samples_tot":"sum","RnaSeq_count":"sum","WGS_count":"sum","WXS_count":"sum","RnaSeq+WGS+WXS":"sum","RnaSeq_only":"sum","DNA_AffyMtrx":"sum","RnaSeq+W*S+DNAffy":"sum"}))

In [45]:
df_4waffle_table2print.loc[len(df_4waffle_table2print.index)]=["sum"]+tmp_list

In [46]:
nw_order=['tissue', 'samples_tot', 'RnaSeq_count', 'WGS_count', 'WXS_count','DNA_AffyMtrx', 'RnaSeq+WGS+WXS', 'RnaSeq_only', 'RnaSeq+W*S+DNAffy']
df_4waffle_table2print_=df_4waffle_table2print.reindex(nw_order,axis=1)

In [47]:
df_4waffle_table2print_.tissue=["Lung","Haematopoietic and lymphoid tissue","Central nervous system","Skin","Largeintestine","Breast","Ovary","Pancreas","Stomach","Upper aerodigestive tract","Kidney","Soft tissue","Bone","Urinary tract","Endometrium","Oesophagus","Liver","Autonomic ganglia","Thyroid","Pleura","Prostate","Biliary tract","Salivary gland","Cervix","Small Intestine","Sum"]

In [48]:
df_4waffle_table2print_

,tissue,samples_tot,RnaSeq_count,WGS_count,WXS_count,DNA_AffyMtrx,RnaSeq+WGS+WXS,RnaSeq_only,RnaSeq+W*S+DNAffy
11,Lung,199,192 [96.48%],98 [49.25%],33 [16.58%],178 [89.45%],14,75,14
7,Haematopoietic and lymphoid tissue,188,176 [93.62%],14 [7.45%],89 [47.34%],180 [95.74%],2,78,2
4,Central nervous system,70,66 [94.29%],15 [21.43%],12 [17.14%],64 [91.43%],1,40,1
18,Skin,61,56 [91.80%],19 [31.15%],17 [27.87%],53 [86.89%],0,21,0
9,Largeintestine,61,59 [96.72%],23 [37.70%],16 [26.23%],56 [91.80%],3,23,2
3,Breast,58,57 [98.28%],35 [60.34%],12 [20.69%],52 [89.66%],1,11,1
13,Ovary,53,47 [88.68%],23 [43.40%],16 [30.19%],53 [100.00%],3,13,3
14,Pancreas,43,41 [95.35%],11 [25.58%],16 [37.21%],41 [95.35%],0,14,0
21,Stomach,37,37 [100.00%],21 [56.76%],16 [43.24%],37 [100.00%],4,4,4
23,Upper aerodigestive tract,32,32 [100.00%],6 [18.75%],18 [56.25%],29 [90.62%],0,8,0


# Get Download List- for Cell Line with complete set of data

In [37]:
df_complete2Download=table_sampleName_1[table_sampleName_1["RnaSeq+W*S+DNAffy"]]

In [38]:
df_complete2Download=df_complete2Download.drop("tmp",axis=1)

## Short

In [43]:
df_complete2Download_short=df_complete2Download.groupby("tissue").first()

In [44]:
list2Drop= list(df_complete2Download_short["Sample_ID"].values)

In [45]:
df_complete2Download_2nd=df_complete2Download[~df_complete2Download.Sample_ID.isin(list2Drop)]

In [46]:
list2Drop2= list(df_complete2Download_2nd["Sample_ID"].values)

In [47]:
df_complete2Download_2nd.shape

(22, 16)

In [74]:
filename = "short_RNA-Seq.txt"
tmp=["",""]
with open(filename, "w") as f:
    for idx,item in df_complete2Download_2nd.iterrows():
        tmp=item["Sample_Name"].split("_")
        #f.write("https://sra-pub-run-odp.s3.amazonaws.com/sra/"+item["SRR_RNA-Seq"]+"/"+item["SRR_RNA-Seq"]+"\t"+tmp[0]+"_"+tmp[1].lower())
        f.write(item["SRR_RNA-Seq"]+"\t"+tmp[0]+"_"+tmp[1].lower())
        f.write("\n")


filename = "short_WGS.txt"
tmp=["",""]
with open(filename, "w") as f:
    for idx,item in df_complete2Download_2nd.iterrows():
        tmp=item["Sample_Name"].split("_")
        #f.write("https://sra-pub-run-odp.s3.amazonaws.com/sra/"+item["SRR_WGS"]+"/"+item["SRR_WGS"]+"\t"+tmp[0]+"_"+tmp[1].lower())
        f.write(item["SRR_WGS"]+"\t"+tmp[0]+"_"+tmp[1].lower())
        f.write("\n")

filename = "short_WXS.txt"
tmp=["",""]
with open(filename, "w") as f:
    for idx,item in df_complete2Download_2nd.iterrows():
        tmp=item["Sample_Name"].split("_")
        #f.write("https://sra-pub-run-odp.s3.amazonaws.com/sra/"+item["SRR_WXS"]+"/"+item["SRR_WXS"]+"\t"+tmp[0]+"_"+tmp[1].lower())
        f.write(item["SRR_WXS"]+"\t"+tmp[0]+"_"+tmp[1].lower())
        f.write("\n")


In [48]:

dfi.export(df_complete2Download_short.style.pipe(make_pretty),"short_list.png")

## ALL RNA

In [54]:
df_allRNA2Download=table_sampleName_1[table_sampleName_1["RNA-Seq"]]

In [56]:
df_allRNA2Download.shape

(1019, 17)

In [61]:
df_allRNA2Download=df_allRNA2Download[~df_allRNA2Download.Sample_ID.isin(list2Drop)]

In [62]:
df_allRNA2Download.shape

(1006, 17)

In [66]:
df_allRNA2Download=df_allRNA2Download[~df_allRNA2Download.Sample_ID.isin(list2Drop2)]

In [67]:
df_allRNA2Download.shape

(984, 17)

In [69]:
filename = "short_RNA-Seq.txt"
tmp=["",""]
with open(filename, "w") as f:
    for idx,item in df_allRNA2Download.iterrows():
        tmp=item["Sample_Name"].split("_")
        #f.write("https://sra-pub-run-odp.s3.amazonaws.com/sra/"+item["SRR_RNA-Seq"]+"/"+item["SRR_RNA-Seq"]+"\t"+tmp[0]+"_"+tmp[1].lower())
        f.write(item["SRR_RNA-Seq"]+"\t"+tmp[0]+"_"+tmp[1].lower())
        f.write("\n")




## ALL WXS

In [44]:
df_allWXS2Download=table_sampleName_1[table_sampleName_1["WXS"]]

In [49]:
df_allWXS2Download=df_allWXS2Download[~df_allWXS2Download.Sample_ID.isin(list2Drop)]

In [50]:
df_allWXS2Download.shape

(313, 17)

In [51]:
filename = "short_WXS.txt"
tmp=["",""]
with open(filename, "w") as f:
    for idx,item in df_allWXS2Download.iterrows():
        tmp=item["Sample_Name"].split("_")
        #f.write("https://sra-pub-run-odp.s3.amazonaws.com/sra/"+item["SRR_WXS"]+"/"+item["SRR_WXS"]+"\t"+tmp[0]+"_"+tmp[1].lower())
        f.write(item["SRR_WXS"]+"\t"+tmp[0]+"_"+tmp[1].lower())
        f.write("\n")


## ALL WGS

In [48]:
df_allWGS2Download=table_sampleName_1[table_sampleName_1["WGS"]]

In [49]:
df_allWGS2Download=df_allWGS2Download[~df_allWGS2Download.Sample_ID.isin(list2Drop)]

In [53]:
df_allWGS2Download

,Unnamed: 0,Sample_ID,tmp,Sample_Name,tissue,cell_line,disease,RNA-Seq,WGS,WXS,RnaSeq+WGS+WXS,RnaSeq_only,DNA_AffyMtrx,RnaSeq+W*S+DNAffy,SRR_RNA-Seq,SRR_WGS,SRR_WXS
22,22,SAMN10988232,"LK2_LUNG,lung,LK-2,carcinoma (squamous_cell_ca...",LK2_LUNG,lung,LK-2,carcinoma (squamous_cell_carcinoma),True,True,False,False,False,True,False,SRR8615840,SRR8652089,none
33,33,SAMN10987646,"SW480_LARGE_INTESTINE,large_intestine,SW480,ca...",SW480_LARGE_INTESTINE,large_intestine,SW480,carcinoma (adenocarcinoma),True,True,False,False,False,True,False,SRR8616059,SRR8670707,none
111,111,SAMN10987999,"NCIH1105_LUNG,lung,NCI-H1105,carcinoma (small_...",NCIH1105_LUNG,lung,NCI-H1105,carcinoma (small_cell_carcinoma),True,True,False,False,False,True,False,SRR8615329,SRR8652138,none
146,146,SAMN10988516,"SCLC21H_LUNG,lung,SCLC-21H,carcinoma (small_ce...",SCLC21H_LUNG,lung,SCLC-21H,carcinoma (small_cell_carcinoma),True,True,False,False,False,True,False,SRR8616096,SRR8670746,none
154,154,SAMN10988044,"HCC1833_LUNG,lung,HCC-1833,carcinoma (adenocar...",HCC1833_LUNG,lung,HCC-1833,carcinoma (adenocarcinoma),True,True,False,False,False,False,False,SRR8616178,SRR8639163,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,1053,SAMN10987782,"NCIH520_LUNG,lung,NCI-H520,carcinoma (squamous...",NCIH520_LUNG,lung,NCI-H520,carcinoma (squamous_cell_carcinoma),True,True,False,False,False,True,False,SRR8615799,SRR8670692,none
1057,1057,SAMN10988018,"SUIT2_PANCREAS,pancreas,SUIT-2,carcinoma (duct...",SUIT2_PANCREAS,pancreas,SUIT-2,carcinoma (ductal_carcinoma),True,True,False,False,False,True,False,SRR8616011,SRR8670709,none
1061,1061,SAMN10988557,"KU1919_URINARY_TRACT,urinary_tract,KU-19-19,ca...",KU1919_URINARY_TRACT,urinary_tract,KU-19-19,carcinoma (transitional_cell_carcinoma),True,True,False,False,False,True,False,SRR8616074,SRR8652072,none
1062,1062,SAMN10987978,"CHAGOK1_LUNG,lung,ChaGo-K-1,carcinoma",CHAGOK1_LUNG,lung,ChaGo-K-1,carcinoma,True,True,False,False,False,True,False,SRR8616079,SRR8639198,none


In [52]:
filename = "short_WGS.txt"
tmp=["",""]
with open(filename, "w") as f:
    for idx,item in df_allWGS2Download.iterrows():
        tmp=item["Sample_Name"].split("_")
        #f.write("https://sra-pub-run-odp.s3.amazonaws.com/sra/"+item["SRR_WXS"]+"/"+item["SRR_WXS"]+"\t"+tmp[0]+"_"+tmp[1].lower())
        f.write(item["SRR_WGS"]+"\t"+tmp[0]+"_"+tmp[1].lower())
        f.write("\n")